# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

In [1]:
import pandas as pd
import geopandas as gpd

RELOAD_DATA = False
eu_data_path = 'datasets/saved/eu_gpd.geojson'
city_data_path = 'datasets/saved/city_data.csv'
sunset_data = 'datasets/saved/sunset_data_2022.geojson'
avg_country_data_path = 'datasets/saved/avg_country.csv'

### EU countries to capitals and GeoPandas data

In [2]:
from sun_data import get_sunset_sunrise_data
from geo_utils import load_eu_countries_as_geopandas, get_eu_city_data, get_avg_country_data

In [3]:
if RELOAD_DATA:
    print('Reloading data from datasets/saved folder ...')
    eu_gpd = gpd.read_file(eu_data_path)
    top_city_data = pd.read_csv(city_data_path)
    sun_data_gpd = gpd.read_file(sunset_data)
    avg_country_data = pd.read_csv(avg_country_data_path)
else:
    print('Generating data ...')
    top_city_data = get_eu_city_data(3)
    top_city_data.to_csv(city_data_path, index=False)

    eu_gpd = load_eu_countries_as_geopandas()
    eu_gpd.to_file(eu_data_path, driver='GeoJSON', index=False)

    avg_country_data = get_avg_country_data(top_city_data, eu_gpd)
    avg_country_data.to_csv(avg_country_data_path, index=False)

    sun_data_gpd = get_sunset_sunrise_data(year=2022, eu_gpd=eu_gpd, csv_path=sunset_data)

Generating data ...


C:\Users\phili\PycharmProjects\CircadianRythmEU\geo_utils.py:30: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\phili\miniconda3\envs\circadian_rythm\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\phili\PycharmProjects\CircadianRythmEU\geo_utils.py:30: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://

In [5]:
eu_gpd.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,iso_a2
43,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-5750519.703 463075.975, -5816...",FR
110,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((1227561.058 8149359.996, 1276642.191...",SE
113,37970874.0,Europe,Poland,POL,595858,"POLYGON ((2614241.130 7153601.786, 2619073.310...",PL
114,8877067.0,Europe,Austria,AUT,445075,"POLYGON ((1890167.860 6127424.985, 1881717.299...",AT
115,9769949.0,Europe,Hungary,HUN,163469,"POLYGON ((2458558.676 6177394.292, 2520364.547...",HU


In [6]:
top_city_data.head()

,population,CODE,country_ISO_A2,NAME,longitude,latitude,mercantor_x,mercantor_y,social_timezone,utc_sun_timezone_offset,summertime_longitudinal_diff_km,longitudinal_diff_km
0,1766746,AT001C,AT,Wien,16.372504,48.208354,1.822579e+06,6.141588e+06,CET,1,1090.199664,-109.800336
1,269997,AT002C,AT,Graz,15.438279,47.070868,1.718581e+06,5.953649e+06,CET,1,1164.937712,-35.062288
2,193814,AT003C,AT,Linz,14.286198,48.305908,1.590332e+06,6.157899e+06,CET,1,1257.104160,57.104160
3,1226329,BE001C,BE,Bruxelles/Brussel,4.351697,50.846557,4.844287e+05,6.594196e+06,CET,1,2051.864240,851.864240
4,530627,BE002C,BE,Antwerpen,4.399708,51.221110,4.897733e+05,6.660499e+06,CET,1,2048.023352,848.023352


In [7]:
sun_data_gpd.head()

,capital,day,month,year,sunrise_UTC,sunset_UTC,geometry
iso_a3,,,,,,,
AUT,Vienna,1,1,2022,6,15,"POLYGON ((1890167.860 6127424.985, 1881717.299..."
AUT,Vienna,2,1,2022,6,15,"POLYGON ((1890167.860 6127424.985, 1881717.299..."
AUT,Vienna,3,1,2022,6,15,"POLYGON ((1890167.860 6127424.985, 1881717.299..."
AUT,Vienna,4,1,2022,6,15,"POLYGON ((1890167.860 6127424.985, 1881717.299..."
AUT,Vienna,5,1,2022,6,15,"POLYGON ((1890167.860 6127424.985, 1881717.299..."


In [8]:
avg_country_data

,social_timezone,utc_sun_timezone_offset,mean_longitudinal_diff_km,mercantor_x,mercantor_y,iso_a2,pop_est,name,pop_norm,dst,weighted_mean_longdiff,norm_weighted_mean_longdiff
0,CET,1,-29.252821,1.822579e+06,6.141588e+06,AT,8877067.0,Austria,0.100071,False,-2.927367,0.001760
1,CET,1,817.999565,4.844287e+05,6.594196e+06,BE,11484055.0,Belgium,0.131666,False,107.702881,0.064764
2,EET,2,373.644509,2.596164e+06,5.266072e+06,BG,6975761.0,Bulgaria,0.077029,False,28.781366,0.017307
3,CET,1,-115.056003,1.605367e+06,6.461437e+06,CZ,10669709.0,Czechia,0.121797,False,-14.013461,0.008427
4,CET,1,267.602771,1.490441e+06,6.894158e+06,DE,83132799.0,Germany,1.000000,False,267.602771,0.160914
5,CET,1,327.866579,1.399294e+06,7.496302e+06,DK,5818553.0,Denmark,0.063004,False,20.656962,0.012421
6,EET,2,275.734280,2.754642e+06,8.275492e+06,EE,1326590.0,Estonia,0.008565,False,2.361567,0.001420
7,CET,1,1250.732949,-4.122809e+05,4.926683e+06,ES,47076781.0,Spain,0.563026,False,704.194795,0.423445
8,EET,2,443.732621,2.776614e+06,8.437122e+06,FI,5520314.0,Finland,0.059390,False,26.353155,0.015847
9,CET,1,865.323856,2.614217e+05,6.250039e+06,FR,67059887.0,France,0.805207,False,696.765077,0.418977


### Generate top n cities per country with timezone features

## Interactive visualization with Panel/Bokeh

In [42]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, CategoricalColorMapper, ColorBar, ColumnDataSource, LabelSet, HoverTool
from bokeh.transform import factor_cmap
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Plasma256
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime, date
import numpy as np

data_field = 'social_timezone'
bokeh_tools = 'wheel_zoom, pan, box_zoom, reset, save'
colorbar_settings = {'title_text_font_size':'12pt','label_standoff':12}

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data):
    p = figure(toolbar_location='right', tools=bokeh_tools, active_scroll ="wheel_zoom",
               title="Distance to eastern timezone meridian for large EU cities",
               x_range=(-1.3*10**6, 4*10**6),
               y_range=(4*10**6,9*10**6))
    p.title.text_font_size = '20px'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # ADD MAP TILES ---------------------------------------------------------------------------
    p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

    # ADD TIMEZONE LINES

    # ADD GEO STUFF FOR COUNTRIES AS A WHOLE -------------------------------------------------
    geo_data_source = get_bokeh_geodata_source(data)

    values = data[data_field]
    palette = brewer['OrRd'][3]
    palette = palette[::-1]
    color_mapper = CategoricalColorMapper(palette = palette, factors=values.unique().tolist())
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title='Timezone', **colorbar_settings)
    country_tz = p.patches('xs','ys', source=geo_data_source,
            fill_color={'field': data_field, 'transform': color_mapper},
            line_color='blue',
            line_width=0.5,
            fill_alpha=0.8)
    p.add_layout(color_bar, 'below')

      # TOOLTIPS FOR COUNTRY PATCHES
    tooltips_country = [
        ('Country', '@iso_a2')
    ]
    p.add_tools(HoverTool(renderers=[country_tz], tooltips=tooltips_country))

    # ===================================================================================================================

    #ADD BARS FOR DISTANCE TO EAST MERIDIAN EFFECT
    length_scale = 200000
    bar_data_source = ColumnDataSource(dict(
            x0=data['mercantor_x'],
            y0=data['mercantor_y'],
            x1=data['mercantor_x'] + (length_scale *  data['norm_weighted_mean_longdiff']) * np.sign(data['weighted_mean_longdiff']),
            y1=data['mercantor_y'],
            long_diff=data['mean_longitudinal_diff_km'],
            weighted_long_diff=data['weighted_mean_longdiff'],
            long_diff_norm=data['norm_weighted_mean_longdiff']
        )
    )
    divider_len = 50000
    divider_data_source = ColumnDataSource(dict(
            x0=data['mercantor_x'],
            y0=data['mercantor_y'] - divider_len / 2,
            x1=data['mercantor_x'],
            y1=data['mercantor_y'] + divider_len / 2,
            text=data['name'],
            text_y=(data['mercantor_y'] + divider_len / 2) + 1000
    ))
    longdiff_quads = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width=5, source=bar_data_source)
    londiff_diviers = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width=2, source=divider_data_source)
    longdiff_label = p.text(x="x0",y="text_y", text="text", source=divider_data_source)

    # TOOLTIPS FOR CITY DATA
    tooltips_longquads = [
        ('Relative distance (km) to timezone border', '@long_diff'),
        ('Weighted (population size) diff. to east timezone border', '@weighted_long_diff'),
        ('Normalized, weighted (population size) effect of distance to east meridian', '@long_diff_norm')
    ]
    p.add_tools(HoverTool(renderers=[longdiff_quads], tooltips=tooltips_longquads, name='long_quads'))
    return p

In [10]:
from bokeh.models import DataTable, TableColumn

def bokeh_country_table(country_data):
    country_data_sorted = country_data.sort_values('norm_weighted_mean_longdiff', ascending=False)
    source = ColumnDataSource(country_data_sorted)
    columns = [
        TableColumn(field='name', title='Country Name'),
        TableColumn(field="iso_a2",title="Country Code (ISO_A2)"),
        TableColumn(field="social_timezone", title="Social Timezone"),
        TableColumn(field="norm_weighted_mean_longdiff", title="Normalized weighted effect of dist. to east meridian"),
        TableColumn(field="mean_longitudinal_diff_km", title="Avg. dist. to east meridian (km)"),
        TableColumn(field="pop_est", title="Estimated population")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [11]:
def bokeh_sun_table(sun_data):
    source = ColumnDataSource(sun_data)

    # Add data table
    columns = [
        TableColumn(field="iso_a3", title="Country Code (ISO_A3)"),
        TableColumn(field="year", title="Year"),
        TableColumn(field="month", title="Month"),
        TableColumn(field="day", title="Day"),
        TableColumn(field="sunrise_UTC", title="Sunrise (UTC/GMT)"),
        TableColumn(field="sunset_UTC", title="Sunrise (UTC/GMT)")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [49]:
def map_visualization():
    # CREATE MAP  ----------------------------------------------------------------------------------
    # Create Map Panel
    map_pane = pn.pane.Bokeh(sizing_mode='scale_both', width_policy='max')
    dst_text = pn.widgets.StaticText(value='Daylight savings time (DST) enabled:')
    dst_toggle = pn.widgets.Switch(name="DST Toggle")

    eu_geo_tz = eu_gpd.merge(avg_country_data)

    def update_map(event):
        eu_geo_tz_filter = eu_geo_tz[eu_geo_tz['dst'] == dst_toggle.value]
        map_pane.object = bokeh_plot_map(eu_geo_tz_filter)
    dst_toggle.param.watch(update_map, 'value')
    dst_toggle.param.trigger('value')

    # CREATE DATATABLES ----------------------------------------------------------------------------------
    sizing_dict = dict(sizing_mode='stretch_both', width_policy='auto', margin=10)
    # Create City Table Panel
    country_data_pane = pn.pane.Bokeh(**sizing_dict)
    country_data_pane.object = bokeh_country_table(avg_country_data)

    # Create Sun Table Panel
    sun_data_pane = pn.pane.Bokeh(**sizing_dict)
    sun_data_pane.object = bokeh_sun_table(sun_data_gpd.iloc[:,:-1])

    # Create panel application layout
    map_vis = pn.Column(pn.Row(dst_text, dst_toggle), map_pane)
    tabs = pn.Tabs(('Map', map_vis), ('Country Data', country_data_pane))
    return tabs

app = map_visualization()

In [50]:
app.show()

Launching server at http://localhost:23171
